In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.sparse import spdiags
from scipy.sparse import diags
from scipy.fftpack import fft2, ifft2
from scipy.integrate import solve_ivp
from scipy.linalg import lu, solve_triangular
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import bicgstab
import scipy.sparse.linalg as spla
from scipy.sparse.linalg import gmres
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FFMpegWriter
from IPython.display import HTML

In [170]:
m = 64
n = m * m
L = 10
x = [-L, L]
y = [-L, L]
delta = (2*L) / m

e0 = np.zeros((n, 1))
e1 = np.ones((n, 1))
e2 = np.copy(e1)
e4 = np.copy(e0)

for j in range(1, m+1):
    e2[m*j-1] = 0
    e4[m*j-1] = 1

e3 = np.zeros_like(e2)
e3[1:n] = e2[0:n-1]
e3[0] = e2[n-1]

e5 = np.zeros_like(e4)
e5[1:n] = e4[0:n-1]
e5[0] = e4[n-1]

diagonals = [e1.flatten(), e1.flatten(), e5.flatten(), 
             e2.flatten(), -4 * e1.flatten(), e3.flatten(), 
             e4.flatten(), e1.flatten(), e1.flatten()]
offsets = [-(n-m), -m, -m+1, -1, 0, 1, m-1, m, (n-m)]

matA = spdiags(diagonals, offsets, n, n).toarray()
matA = matA/(delta**2)

diagonals = [e1.flatten(), -1 * e1.flatten(), e1.flatten(), -1 * e1.flatten()]
offsets = [-n+m, -m, m, n-m]

matB = spdiags(diagonals, offsets, n, n).toarray()
matB = matB / (2* delta)

diagonals = [e1.flatten(), -e1.flatten()]
offsets = [1, -1]

matC = spdiags(diagonals, offsets, n, n).toarray()

for i in range(m):
    matC[(i + 1)*m - 1, i *m] = 1
    matC[i*m, i*m+(m-1)] = -1
    matC[i*m, i*m - 1] = 0
    matC[i*m - 1, i*m] = 0

matC = matC / (2* delta)

In [171]:
def initial_conditions(case):
    if case == "original":
        return np.exp(-X**2 - Y**2/20)
    elif case == "opposite_gaussians":
        return np.exp(-((X + 5)**2 + Y**2/20)) - np.exp(-((X - 5)**2 + Y**2/20))
    elif case == "same_gaussians":
        return np.exp(-((X + 5)**2 + Y**2/20)) + np.exp(-((X - 5)**2 + Y**2/20))
    elif case == "colliding_pairs":
        return (np.exp(-((X + 5)**2 + (Y + 5)**2/20)) - np.exp(-((X - 5)**2 + (Y + 5)**2/20)) +
                np.exp(-((X + 5)**2 + (Y - 5)**2/20)) - np.exp(-((X - 5)**2 + (Y - 5)**2/20)))
    elif case == "random_vortices":
        vortices = np.zeros_like(X)
        for _ in range(15):
            x0, y0 = np.random.uniform(-10, 10, 2)
            amp = np.random.choice([-1, 1]) * np.random.uniform(0.5, 1.5)
            ellipticity = np.random.uniform(0.8, 1.2)
            vortices += amp * np.exp(-((X - x0)**2/20 + (ellipticity * (Y - y0))**2/20))
        return vortices

A = matA
B = matB
C = matC

tspan = np.arange(0, 4+0.5, 0.5)
nu = 0.001
Lx, Ly = 20, 20
nx, ny = 64, 64
N = nx * ny

# Define spatial domain and initial conditions
x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)
w0 = initial_conditions("original")

# Define spectral k values
kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

In [172]:
def spc_rhs(t, w0, nx, ny, K, nu):
    w2d = w0.reshape((nx, ny))
    wt = fft2(w2d)
    psit = -wt / K
    psi = np.real(ifft2(psit))
    psi = psi.flatten()
    rhs = (nu * A.dot(w0) + B.dot(w0) * C.dot(psi) - C.dot(w0) * B.dot(psi))
    return rhs

w0 = w0.flatten()
wtsol = solve_ivp(spc_rhs, [0, 4], w0, method= "RK45", t_eval=tspan, args=(nx, ny, K, nu))
A1 = wtsol.y
    
# ================= Animation ======================

# fig, ax = plt.subplots()
# c = ax.pcolor(X, Y, A1[-1], shading='auto', cmap='viridis')
# fig.colorbar(c, ax=ax)
# time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes, color='white', fontsize=12)

# # Define the update function for the animation
# def update(frame):
#     c.set_array(A1[frame].flatten())
#     time_text.set_text(f'Time: {tspan[frame]:.2f}')
#     return c, time_text

# # Create the animation
# ani = FuncAnimation(fig, update, frames=len(A1), interval=100, blit=False)

# plt.close(fig)
# HTML(ani.to_jshtml())
# ani.save('fft_random_vortices.gif', writer = 'Pillow', fps = 10)

In [173]:
# A/b
A[0, 0] = 2
start_time = time.time()
def a_b_rhs(t, w0, nu):
    psi = np.linalg.solve(A, w0)
    rhs = (nu * A.dot(w0) + B.dot(w0) * C.dot(psi) - C.dot(w0) * B.dot(psi))
    return rhs

w = np.exp(-X**2 - Y**2 / 20)
w0 = w.flatten()
wtsol = solve_ivp(a_b_rhs, [0, 4], w0, method= "RK45", t_eval=tspan, args=(nu,))
wtsol = wtsol.y
end_time = time.time()
elapsed_time = end_time - start_time
print (f"Elapsed time for A/b: {elapsed_time:.2f} seconds")
A2 = wtsol

Elapsed time for A/b: 11.14 seconds


In [174]:
# LU Decomp
start_time = time.time()
P, L, U = lu(A)
def LU_rhs(t, w0, nu):
    Pb = np.dot(P, w0)
    y = solve_triangular(L, Pb, lower=True)
    psi = solve_triangular(U, y, lower=False)
    rhs = (nu * A.dot(w0) + B.dot(w0) * C.dot(psi) - C.dot(w0) * B.dot(psi))
    return rhs

w = np.exp(-X**2 - Y**2 / 20)
w0 = w.flatten()
wtsol = solve_ivp(LU_rhs, [0, 4], w0, method= "RK45", t_eval=tspan, args=(nu,))
wtsol = wtsol.y
end_time = time.time()
elapsed_time = end_time - start_time
print (f"Elapsed time for LU Decomp: {elapsed_time:.2f} seconds")
A3 = wtsol

Elapsed time for LU Decomp: 1.56 seconds


In [175]:
# BICGSTAB
start_time = time.time()
A_BIC = csc_matrix(A)
def BICG_rhs(t, w0, nu):
    psi, exit_code = bicgstab(A_BIC, w0, atol=1e-8)
    rhs = (nu * A_BIC.dot(w0) + B.dot(w0) * C.dot(psi) - C.dot(w0) * B.dot(psi))
    return rhs

w = np.exp(-X**2 - Y**2 / 20)
w0 = w.flatten()
wtsol = solve_ivp(BICG_rhs, [0, 4], w0, method= "RK45", t_eval=tspan, args=(nu,))
wtsol = wtsol.y
end_time = time.time()
elapsed_time = end_time - start_time
print (f"Elapsed time for BICGSTAB Decomp: {elapsed_time:.2f} seconds")
A4 = wtsol

Elapsed time for BICGSTAB Decomp: 1.41 seconds


In [176]:
# GMRES
start_time = time.time()
A_GMR = csc_matrix(A, dtype=float)
B_GMR = csc_matrix(B, dtype=float)
C_GMR = csc_matrix(C, dtype=float)
def GMRES_rhs(t, w0, nu):
    psi, exit_code = gmres(A_GMR, w0, atol=1e-8)
    rhs = (nu * A_GMR.dot(w0) + B_GMR.dot(w0) * C_GMR.dot(psi) - C_GMR.dot(w0) * B_GMR.dot(psi))
    return rhs

w = np.exp(-X**2 - Y**2 / 20)
w0 = w.flatten()
wtsol = solve_ivp(GMRES_rhs, [0, 4], w0, method= "RK45", t_eval=tspan, args=(nu,))
wtsol = wtsol.y
end_time = time.time()
elapsed_time = end_time - start_time
print (f"Elapsed time for GMRES Decomp: {elapsed_time:.2f} seconds")
A5 = wtsol

Elapsed time for GMRES Decomp: 10.79 seconds


In [177]:
print("A1:", A1)
print("A2:", A2)
print("A3:", A3)

A1: [[2.50656748e-46 3.54964961e-45 1.85768096e-44 ... 1.85272739e-42
  4.67439366e-42 1.11058944e-41]
 [1.17762859e-43 6.53319004e-43 2.62245678e-42 ... 1.58946594e-40
  3.56498221e-40 7.46793534e-40]
 [4.55107657e-41 1.92934361e-40 6.43507373e-40 ... 2.49607447e-38
  5.08478333e-38 9.69634778e-38]
 ...
 [1.96785570e-38 1.23357409e-37 5.47591944e-37 ... 4.35250271e-35
  1.02347073e-34 2.23043626e-34]
 [6.19028421e-41 5.33777602e-40 2.93883953e-39 ... 3.67992748e-37
  9.45792605e-37 2.23822117e-36]
 [1.60178709e-43 1.99380943e-42 1.41671738e-41 ... 2.79962856e-39
  7.95937714e-39 2.07748843e-38]]
A2: [[ 2.50656748e-46 -1.78784704e-36  1.02706391e-36 ... -5.76806489e-26
  -6.68287054e-25  3.02050167e-24]
 [ 1.17762859e-43 -2.26093387e-29  1.85614646e-29 ... -1.48306461e-19
  -7.82424220e-19  1.67449651e-18]
 [ 4.55107657e-41 -4.24661881e-29  3.48720993e-29 ... -2.20165937e-20
  -2.91637720e-19  1.23961335e-18]
 ...
 [ 1.96785570e-38  7.20814729e-28 -6.58759786e-28 ...  4.01674620e-20
  